In [1]:
%load_ext autoreload
%autoreload 2
import ast
import openai
import re
import os
# openai.api_key = os.getenv('OPENAI_KEY')

from document import *

In [2]:
file_path = './app.py'

In [3]:
functions = extract_functions(file_path)

In [4]:
function = functions[5]

In [6]:
docstring = generate_docstring(ast.unparse(function))

In [7]:
docstring

'"""\nDownload and display recipe information scraped from a user-provided URL. If the recipe already exists in the master recipe dictionary, the user is prompted to either update or replace the existing recipe.\n\n    Parameters:\n    none\n\n    Returns:\n    none\n\n    Raises:\n    StopIteration: if an invalid or empty URL is provided\n\n"""'

In [11]:
add_docstrings_to_file(file_path)

update_page
"""
Updates the selected page in the Streamlit app.

Arguments:
page_name -- The name of the page to switch to.

Side Effects:
Updates the Streamlit session state with the selected page.

Returns:
None
""""""



SyntaxError: EOF while scanning triple-quoted string literal (<unknown>, line 12)